# `GroupBy`, `agg` (aggregrate), 'orderBy' operations
### Dr. Tirthajyoti Sarkar, Fremont, CA 94536
In this notebook, we will illustrate the `groupBy`, agggreate (`agg`), and `orderBy` operations that can be performed on a DataFrame object.<br>
We will also show the use of standard functions that can be imported. 

### [Spark dataframe guide](https://spark.apache.org/docs/2.2.0/sql-programming-guide.html)


### Creating SparkSession and build an app

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark1 = SparkSession.builder.appName('aggs').getOrCreate()

### Read a dataset from a CSV file into a dataframe

In [3]:
df = spark1.read.csv('Data/sales_info.csv',inferSchema=True,header=True)

### See how the data looks like and print the schema

In [4]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



### `groupBy`

In [6]:
df.groupby('Company')

#### We can perform a variety of operations on this `pyspark.sql.group.GroupedData` object. 
For example, taking a `mean` (average).

In [7]:
df.groupby('Company').mean()

DataFrame[Company: string, avg(Sales): double]

So, the operation returns a DataFrame, not the actual mean. **Lazy evaluation** at play.

In [8]:
df.groupby('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



#### Some other operations
* `max`
* `min`
* `count`

In [9]:
df.groupby('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [10]:
df.groupby('Company').min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [11]:
df.groupby('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



### `agg` or aggregrating
This method works with a slightly unconventional syntax. We have to pass on a dictionary as argument.<br>
Suppose, we want the sum total of sales. We use `sum` but pass it as the value of the key `Sales` of a dictionary, 

In [12]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



Suppose we want the maximum of all sales

In [13]:
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



### Combine `groupBy` and `agg` to get info by company

In [14]:
group_data = df.groupBy('Company')

In [15]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



### Import and use other functions

In [16]:
from pyspark.sql.functions import stddev,countDistinct,count,avg

In [17]:
df.agg({'Sales':'stddev'}).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



#### We could compute the same quantity using simple `select` too

In [18]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



#### We can count how many total sales and how many of them are distinct in value

In [19]:
df.select(count('Sales')).show()

+------------+
|count(Sales)|
+------------+
|          12|
+------------+



In [20]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



### Formatting results
Many of these functions and mathematical operations produce results with long number format. We can change the format suitably using `format_number` function. We can also change the long column name to something shorter using `alias` method.

In [21]:
from pyspark.sql.functions import format_number

In [22]:
sales_std = df.select(stddev('Sales').alias('std'))

In [23]:
sales_std

DataFrame[std: double]

In [24]:
sales_std=sales_std.select(format_number('std',2).alias('final'))

In [25]:
sales_std

DataFrame[final: string]

In [26]:
sales_std.show()

+------+
| final|
+------+
|250.09|
+------+



### Use `orderBy` to order a particular column by value

In [27]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [28]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



#### To order by descending values, it is bit more involved. You have to pass on the actual column object (not just the name as a string) and call `desc` method on it.

In [29]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

